<a href="https://colab.research.google.com/github/tantowijh/colab-collection/blob/main/Downloader_to_Google_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <font color="red">**Special Tools**</font>
<font color="blue">Run the following command to start renaming or see video information:

In [ ]:
#@title **<font color="green">Renaming Files**
import os
import signal
import sys

def sigint_handler(signal, frame):
    print ('KeyboardInterrupt is caught')
    sys.exit(0)
signal.signal(signal.SIGINT, sigint_handler)

InputLocation = "" #@param {type:"string"}

Text2Replace  = "" #@param {type:"string"}
ReplacementText   = "" #@param {type:"string"}

FileType = "mkv" #@param ["Write it yourself...", "mp4", "mkv", "png", "jpg", "jpeg", "ass"]
if FileType == "Write it yourself...":
  ftype = input('Enter the file type to rename: ') 
  Extension = '.' + ftype
else:
  Extension = '.' + FileType

PutLabel = False #@param {type:"boolean"}
Label = "" #@param {type:"string"} 

fix = len(Extension)
#-(int(fix))

# Renaming the file

def RenameMe():
  for count, filename in enumerate(os.listdir(InputLocation)):
    if(filename.endswith(Extension)):
          src =f"{InputLocation}/{filename}"  # foldername/filename, if .py file is outside folder
          cuk =f"{filename}"
          fname = cuk.replace(Text2Replace, ReplacementText)
          dst =f"{InputLocation}/{fname}"
          
          # rename() function will
          # rename all the files with the pre defined extension
          os.rename(src, dst)


def LabelMe():
  for count, filename in enumerate(os.listdir(InputLocation)):
    if(filename.endswith(Extension)):
          src =f"{InputLocation}/{filename}"  # foldername/filename, if .py file is outside folder
          dst =f"{InputLocation}/{Label} {filename}"
          
          # rename() function will
          # rename all the files with the pre defined extension
          os.rename(src, dst)

if PutLabel == False:
  RenameMe()
else:
  LabelMe()

In [ ]:
import os
import re
from pathlib import Path
!sudo apt-get install mkvtoolnix &> /dev/null
logsss = '.logfiles'
logss = os.path.exists(logsss)
if not logss:
  os.makedirs(logsss)

#@markdown <h3><b><font color="green">Generate Information from Video:</b></h3>
#@markdown <h5><b><i><font color="blue">Put file name (include the extension for single file). Put directory path (for batch file).</i></b></h5>

#@markdown - - -

InputLocation = ""  # @param {type:"string"}
InputFormat = ".mkv" #@param [".mp4", ".mov", ".mkv", ".avi", ".ts", ".flv", ".webm", ".wmv", ".mpg", ".m4v", ".f4v", ".m2ts", ".mpeg", ".3gp", ".MP4", ".MOV", ".MKV", ".AVI", ".TS", ".FLV", ".WEBM", ".WMV", ".MPG", ".M4V", ".F4V", ".M2TS", ".MPEG", ".3GP"]
SearchForSubfolder = False #@param {type:"boolean"}
#@markdown <h5>&ensp;&ensp;<font color="red">Check search for subfolder will make it search the files inside the <b>subdirectory</b> where you provide <b>path location</b>.

#@markdown <br>
#@markdown <h5><b><i><font color="blue">Which option you want to generate information.</i></b></h5>

#@markdown - - -
Audio = False  # @param {type:"boolean"}

Subtitle = True  # @param {type:"boolean"}

Attachment = False  # @param {type:"boolean"}

if SearchForSubfolder:
    ext = '**/*' + InputFormat
else:
    ext = '*' + InputFormat


def detect_available_audio_tracks(source_video):

    if source_video.endswith(InputFormat):
        file = os.popen(rf'"mkvmerge" -i "{source_video}"').read()
        AvailableAudioTracks = re.findall(
            r"Track ID ([0-9]+): audio \((.+)\)", file, re.MULTILINE)
        Filename = re.findall(r"File '(.+)'", file)
        audio_tracks = ''
        for track in AvailableAudioTracks:
            audio_tracks += f'\033[1;0mAudio track {track[0]} \033[1;31mformat \033[1;32mtype \033[1;34m{track[1]}\n'

        if audio_tracks:

            print(
                    f'\n\033[1;33mThese audios were found in the file {Filename[0]} \n{audio_tracks}')

    elif os.path.isdir(source_video):
        for source_video in Path(source_video).glob(ext):
            file = os.popen(rf'"mkvmerge" -i "{source_video}"').read()
            AvailableAudioTracks = re.findall(
                r"Track ID ([0-9]+): audio \((.+)\)", file, re.MULTILINE)
            Filename = re.findall(r"File '(.+)'", file)
            audio_tracks = ''
            for track in AvailableAudioTracks:
                audio_tracks += f'\033[1;0mAudio track {track[0]} \033[1;31mformat \033[1;32mtype \033[1;34m{track[1]}\n'

            if audio_tracks:

                print(
                        f'\n\033[1;33mThese audios were found in the file {Filename[0]} \n{audio_tracks}')

    else:
        print('error')


def detect_available_subtitle_tracks(source_video):

    if source_video.endswith(InputFormat):
        !ffprobe '{source_video}' 2>&1 >/dev/null | grep Stream | grep Subtitle > ".logfiles/subtitles.txt"
        subtitles = '.logfiles/subtitles.txt'
        print("\033[1;33mThese subtitles are found in the file ", source_video)
        with open(subtitles,"r") as logs:
          count = 0
          for l in logs:
            res = re.search('(\(.+?\)):', l).group(1)
            l = l.strip()
            src =f"{l}"
            out = src.replace("Stream #0:", "\033[1;0mSubtitle stream pos ")
            out = out.replace(res+':', "")
            out = out.replace('Subtitle:', "\033[1;31mtrack \033[1;34m" + str(count) + "\033[1;32m language" + "\033[1;34m" + res + "\033[1;0m format type\033[1;31m")
            count += 1
            print(out)
          print("\n")

    elif os.path.isdir(source_video):
        for source_video in Path(source_video).glob(ext):
            !ffprobe '{source_video}' 2>&1 >/dev/null | grep Stream | grep Subtitle > ".logfiles/subtitles.txt"
            subtitles = '.logfiles/subtitles.txt'
            print("\033[1;33mThese subtitles are found in the file ", source_video)
            with open(subtitles,"r") as logs:
              count = 0
              for l in logs:
                res = re.search('(\(.+?\)):', l).group(1)
                l = l.strip()
                src =f"{l}"
                out = src.replace("Stream #0:", "\033[1;0mSubtitle stream pos ")
                out = out.replace(res+':', "")
                out = out.replace('Subtitle:', "\033[1;31mtrack \033[1;34m" + str(count) + "\033[1;32m language" + "\033[1;34m" + res + "\033[1;0m format type\033[1;31m")
                count += 1
                print(out)
              print("\n")

    else:
        print('error')


def detect_available_attachment_tracks(source_video):

    if source_video.endswith(InputFormat):
        file = os.popen(rf'"mkvmerge" -i "{source_video}"').read()
        AvailableAttachment = re.findall(
            r"Attachment ID ([0-9]+):.+ file name '(.+)'", file, re.MULTILINE)
        Filename = re.findall(r"File '(.+)'", file)
        Attachment = ''
        for att in AvailableAttachment:
            Attachment += f'\033[1;0mAttachment track {att[0]} \033[1;32mname \033[1;34m{att[1]}\n'

        if Attachment:

            print(
                    f'\n\033[1;33mThese attachments were found in the file {Filename[0]} \n{Attachment}')

    elif os.path.isdir(source_video):
        for source_video in Path(source_video).glob(ext):
            file = os.popen(rf'"mkvmerge" -i "{source_video}"').read()
            AvailableAttachment = re.findall(
                r"Attachment ID ([0-9]+):.+ file name '(.+)'", file, re.MULTILINE)
            Filename = re.findall(r"File '(.+)'", file)
            Attachment = ''
            for att in AvailableAttachment:
                Attachment += f'\033[1;0mAttachment track {att[0]} \033[1;32mname \033[1;34m{att[1]}\n'

            if Attachment:

                print(
                        f'\n\033[1;33mThese attachments were found in the file {Filename[0]} \n{Attachment}')

    else:
        print('error')


if Audio:
    detect_available_audio_tracks(InputLocation)

if Subtitle:
    detect_available_subtitle_tracks(InputLocation)

if Attachment:
    detect_available_attachment_tracks(InputLocation)


## <font color="red">**Mount Google Drive**</font>
<font color="blue">Run the following command to mount google drive:

In [ ]:
#@title **<font color="green">Link your Google Drive**
#@markdown <center></center>
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#@title **<font color="green">Run this code if you want a longer run time (Optional but recommended)</font>**
#@markdown *This cell runs a JS code that automatically reconnects you to avoid inactivity due to idle.*

import IPython
from IPython.display import clear_output
from google.colab import output

display(IPython.display.Javascript('''
 function connectRefresher() {
       window.ConnectButtonIntervalId = setInterval(function ConnectButton(){
                console.log("connected"); 
                document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
                document.querySelector("colab-sessions-dialog").shadowRoot.querySelector("#footer > div > paper-button").click();
                console.log("closed the dialog!!"); 
            },60000);
    }
    
 function clearRefresher() { 
           console.log("clear Interval called !!");
           clearInterval(window.ConnectButtonIntervalId);
    }

 connectRefresher(); //to connect the refresher
 clearRefresher(); //to disconnect the refresher
'''))

clear_output()

## <font color="red">**Google Drive Downloader**</font>
<font color="blue">Run the following command to download files from <font color="green">google drive.

In [ ]:
#@title **<font color="green">Download From Google Drive**
#@markdown <font color="blue">Input file name that you want to <font color="red">**save as**</font> and consider adding an <font color="red">**extension file type**</font> at the end, then put the URL and download location.</font>

import re, os
from IPython.display import clear_output

LargeFile = True #@param {type:"boolean"}
FileNametoSave = "" #@param {type:"string"}
GoogleSharedLink = "" #@param {type:"string"}
# Match six-letter word
pattern = r"[-\w]{18,}"
# match() method
result = re.match(pattern, GoogleSharedLink)
# Output None
# search() method
result = re.search(pattern, GoogleSharedLink)
fileId = result.group()
# findall() method
result = re.findall(pattern, GoogleSharedLink)

# Define download location
DownloadLocation = "/content" #@param {type:"string"}

from pathlib import Path
Path(DownloadLocation).mkdir(parents=True, exist_ok=True)

%cd '{DownloadLocation}'
clear_output()

if LargeFile == False:
  !wget -q --show-progress --progress=bar:force --no-check-certificate 'https://docs.google.com/uc?export=download&id={fileId}' -O '{FileNametoSave}'
else:
  !wget -q --show-progress --progress=bar:force --load-cookies '/tmp/cookies.txt' "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate "https://docs.google.com/uc?export=download&id={fileId}" -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={fileId}" -O '{FileNametoSave}' && rm -rf /tmp/cookies.txt



## <font color="red">**Git Downloader**</font>
<font color="blue">Run the following command to download files from <font color="green">git or github.

In [ ]:
# @title **<font color="green">Download from Git**

import os
import subprocess

#@markdown <font color="blue">Put the URL and download location.</font>
Download_Location = "" #@param {type:"string"}
Download_Link = "/content" #@param {type:"string"}

%cd '{Download_Location}' 
!git clone '{Download_Link}' 

## <font color="red">**Direct File Downloader**</font>
<font color="blue">Run the following command to download files from <font color="green">internets.

In [ ]:
#@markdown <font color="green"><b>Click here to upload your txt file from your local computer!</b></font>

from google.colab import files
uploaded = files.upload()

filenames = uploaded.keys()
for file in filenames:
    data = uploaded[file]

In [ ]:
from sys import path
#@title <font color="green">**Direct File download**</font>
#@markdown <font color="blue"><font color="red">Wget Method</font> the files needs to be of some kind of extension or something similar, <font color="green"><i>e.x. .exe .zip .apk etc.</i></font>
#@markdown <font color="red">Aria2 Method</font> you can see it here <font color="green"><i>aria2.github.io</i></font>. You can also stop seeding immediately for torrent download.
#@markdown - - -
NoSeeding = False #@param {type:"boolean"}
DownloadMethod = "ARIA2" #@param ["WGET", "ARIA2"]

DownloadOption = "Single Download" #@param ["Single Download", "Multiple Download"]
Download_Location = "/content/drive/Shareddrives/06. YT-DLP" #@param {type:"string"}
if Download_Location:
  if not Download_Location.endswith('/'):
    Download_Location = Download_Location + '/'


#@markdown &ensp;&ensp;<font color="blue">For <font color="gren">Multi Downloads ></font><font color="red"> provide txt file path</font> containing batch URLs, separated by new line in the</font> <font color="gren">DownloadURL.</font>
Download_Link = "" #@param {type:"string"}

import os
import subprocess

from os import path
from IPython.utils import io

if DownloadMethod == "ARIA2":
  if not path.exists("/usr/local/bin/aria2c"):
    with io.capture_output() as captured:
      !sudo apt install aria2


def wget():
  def SingleDwnld():
    if "?" in Download_Link:
      filtering = "/".join(Download_Link.split("/")[Download_Link.count('/'):])
      before, sparator, after = filtering.partition('?')
      if len(before) > 0:
        name = before
      !wget '{Download_Link}' -O '{Download_Location}{name}'
    else:
      !wget '{Download_Link}'

  def MultipleDwnld():
    if Download_Location:
      %cd '{Download_Location}'
      !wget -i '{Download_Link}'
      with io.capture_output() as captured:
        !cd '/content'
    else:
      !wget -i '{Download_Link}'

  if DownloadOption == "Single Download":
    SingleDwnld()
  if DownloadOption == "Multiple Download":
    MultipleDwnld()
def aria2():
  def SingleDwnld():
    %cd '{Download_Location}'
    if NoSeeding:
      !aria2c --seed-time=0 '{Download_Link}' --file-allocation=none -c --auto-file-renaming=false
    else:
      !aria2c '{Download_Link}' --file-allocation=none -c --auto-file-renaming=false
    with io.capture_output() as captured:
      !cd '/content'

  def MultipleDwnld():
    %cd '{Download_Location}'
    if NoSeeding:
      !aria2c -i '{Download_Link}' --seed-time=0 --file-allocation=none -c --auto-file-renaming=false
    else:
      !aria2c -i '{Download_Link}' --file-allocation=none -c --auto-file-renaming=false
    with io.capture_output() as captured:
      !cd '/content'

  if DownloadOption == "Single Download":
    SingleDwnld()
  if DownloadOption == "Multiple Download":
    MultipleDwnld()
if DownloadMethod == "WGET":
  wget()
if DownloadMethod == "ARIA2":
  aria2()


## <font color="red">**Zippyshare Downloader**</font>
<font color="blue">Run the following command to download files from <font color="green">zippyshare.

In [ ]:
#@markdown <font color="green"><b>Click here to upload your txt file from your local computer!</b></font>

from google.colab import files
uploaded = files.upload()

filenames = uploaded.keys()
for file in filenames:
    data = uploaded[file]

In [ ]:
import os
from IPython.utils import io
from IPython.display import HTML, clear_output


if not os.path.exists("/usr/share/plowshare"):
  with io.capture_output() as captured:
    %cd /content
    !git clone https://github.com/mcrapet/plowshare.git
    %cd plowshare
    !make install
    !plowmod --install
    !apt-get install nodejs
    %cd ..
    %rm -rf plowshare/

#@title <h3><b><font color="green">Click Here to START Download</b></h3>
#@markdown <font color="blue">For <font color="gren">Multi Downloads ></font><font color="red"> provide txt file path</font> containing batch URLs, separated by new line in the</font> <font color="gren">DownloadURL.</font>
#@markdown - - -
DownloadMethod = "Single Download" #@param ["Single Download", "Multi Downloads"]
DownloadURL = "https://www18.zippyshare.com/d/JN8CMwu5/507320/%5bLENDRIVE%5d_Tensei_Kenja_-_08_%5b1080p_HEVC_10bit%5d%5bDualsubs%5d.mkv" #@param {type:"string"}
DownloadLocation = "/content" #@param {type:"string"}

if DownloadMethod == "Single Download":
  !plowdown "$DownloadURL" -o "$DownloadLocation"
if DownloadMethod == "Multi Downloads":
  !plowdown "$DownloadURL" -m -o "$DownloadLocation"

## <font color="red">**Download a Website**</font>
<font color="blue">Run the following command to download the website recursively:

```
wget -r --no-parent http://www.example.com
```

<font color="blue">This will download the pages without altering their HTML source code.

<font color="blue">When you want to change the links on the pages automatically to point to the downloaded files then use this command instead:

```
wget -r --convert-links --no-parent http://www.example.com
```

<font color="blue">When all HTML files shall get a .html extension, then add the "--html-extension" option.

```
wget -r --convert-links --html-extension --no-parent http://www.example.com
```

In [ ]:
#@title **[3] Direct File download**
#@markdown *Download website option:*

import os
import subprocess

ChooseOption = "3. Download the pages and change the links on the pages automatically to point to the downloaded files with HTML Extension." #@param ["1. Download the pages without altering their HTML source code.", "2. Download the pages and change the links on the pages automatically to point to the downloaded files.", "3. Download the pages and change the links on the pages automatically to point to the downloaded files with HTML Extension."]

def option1():
  Download_Location = input("Enter location you want to download the website files: ")
  Download_Link = input("Enter website link you want to download: ")
  %cd '{Download_Location}'
  !wget -r --no-parent '{Download_Link}'
  
def option2():
  Download_Location = input("Enter location you want to download the website files: ")
  Download_Link = input("Enter website link you want to download: ")
  %cd '{Download_Location}'
  !wget -r --convert-links --no-parent '{Download_Link}'

def option3():
  Download_Location = input("Enter location you want to download the website files: ")
  Download_Link = input("Enter website link you want to download: ")
  %cd '{Download_Location}'
  !wget -r --convert-links --html-extension --no-parent '{Download_Link}'

if ChooseOption == "1. Download the pages without altering their HTML source code.":
  option1()
if ChooseOption == "2. Download the pages and change the links on the pages automatically to point to the downloaded files.":
  option2()
if ChooseOption == "3. Download the pages and change the links on the pages automatically to point to the downloaded files with HTML Extension.":
  option3()